# 專題（二）：建置Bert新聞分類器之資料集

## 專案目標
- 目標：請試著建製 BertForSequenceClassification 看得懂的資料集 NewsDataset
- news_clustering_train.tsv 中有 1800 篇新聞，六種類別的新聞各 300 篇
- news_clustering_test.tsv 中有 600 篇新聞，六種類別的新聞各 100 篇
- 六種類別：體育、財經、科技、旅遊、農業、遊戲

## 實作提示
- STEP1：從 news_clustering_train.tsv 和 news_clustering_test.tsv 中取出標題和類別
- STEP2：繼承 torch.utils.data.Dataset 並實作 NewsDataset，其中需要用到 bert tokenizer (請參考官方對BertForSequenceClassification的說明)
- STEP3：因為每一個從 NewsDataset 來的樣本長度都不一樣，所以需要實作 collate_fn，來zero padding 到同一序列長度
- STEP4：使用 torch.utils.data.DataLoader 來創造 train_loader和valid_loader

## 重要知識點：專題結束後你可以學會
- 如何讀取並處理 NLP 資料，產生可以適用 BertForSequenceClassification 的資料集
- 了解 BERT 的 Sequence Classification 任務如何進行

In [ ]:
# 連接個人資料 讀取 ＰＴＴ 訓練資料和儲存模型
#先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/project_1_2/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/project_1_2


In [ ]:
!python --version

Python 3.7.11


Python 3.6.9

In [ ]:
!pip install torch
!pip install transformers
#!pip install -q transformers
# 設定 torchtext 版本 安裝完必須重新啟動執行階段
!pip install torchtext==0.6.0

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [ ]:
df_train[:5]

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍


In [ ]:
len(df_train)

1800

In [ ]:
df_test.head(5)

,index,class,title
0,1800,體育,如果騎士火箭進入總決賽，誰的勝算大？
1,1801,體育,從個人競技狀態來看，三個階段的詹姆斯，哪個最強？
2,1802,體育,騎士總冠軍！地球人誰能阻擋詹姆斯？史上最佳就是他！打服所有人
3,1803,體育,詹姆斯絕殺，騎士3比0，猛龍懷疑人生
4,1804,體育,騎士和步行者戰成搶七險勝，而猛龍即將被橫掃，步行者跟猛龍的區別在哪裡？


In [ ]:
len(df_test)

600

In [ ]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

valid_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
valid_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

train_titles:

{0: '亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列',
 
 1: '9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜',
 
 2: '如果今年勇士奪冠，下賽季詹姆斯何去何從？',
 
 3: '超級替補！科斯塔本賽季替補出場貢獻7次助攻',
 
 4: '騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍',
 
 5: '如果朗多進入轉會市場，哪些球隊適合他？',

}

In [ ]:
train_titles[0]

'亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列'

In [ ]:
len(train_titles)

1800

train_classes:

{0: '體育',

 1: '體育',

 2: '體育',

 3: '體育',

 4: '體育',

 5: '體育',

}

In [ ]:
train_classes[0]

'體育'

In [ ]:
len(valid_titles)

600

In [ ]:
ALL_NEWS_CLASSES = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

In [ ]:
MODEL_NAME = 'bert-base-chinese'

In [ ]:
# 建立數據集
class NewsDataset(Dataset):
    def __init__(self, tokenizer, titles, classes):
        self.tokenizer = tokenizer
        self.indexes = []
        self.texts = []
        self.labels = []
        for index in titles:
            self.indexes.append(index)
            self.texts.append(titles[index])
            self.labels.append(classes[index])

    def __getitem__(self, idx):
        text = self.texts[idx]

        input = self.tokenizer(text, return_tensors='pt')
        label = torch.tensor(ALL_NEWS_CLASSES.index(self.labels[idx]))

        return input, label

    def __len__(self):
        return len(self.indexes)


def create_mini_batch(samples):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    labels = []
    for s in samples:
        input_ids.append(s[0]['input_ids'].squeeze(0))
        token_type_ids.append(s[0]['token_type_ids'].squeeze(0))
        attention_mask.append(s[0]['attention_mask'].squeeze(0))
        labels.append(s[1])

    # zero pad 到同一序列長度
    # Code Here
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    token_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=0)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    # End
 
    labels = torch.stack(labels)

    return input_ids, token_type_ids, attention_mask, labels

In [ ]:
batch_size = 32

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)  # Code Here

train_dataset = NewsDataset(tokenizer, train_titles, train_classes)
valid_dataset = NewsDataset(tokenizer, valid_titles, valid_classes)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    collate_fn=create_mini_batch,
    shuffle=True)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    collate_fn=create_mini_batch)